In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
TrustPilot = pd.read_csv('TrustPilot_2.csv')
TrustedShop = pd.read_csv('TrustedShop_2.csv')

### Application du GradientBoostingClassifier sur le df TrustPilot

In [12]:
TrustedShop.head(2)

,Unnamed: 0,Unnamed: 0.1,Commentaire,star,date,client,reponse,source,company,ville,ecart,YearMonth,year,comments_filter
0,0,2320,Excelente,5,2021-06-08,NaN,"¡Hola , Gracias por compartir tu experiencia c...",TrustedShop,ShowRoom,NaN,14.0,2021-06,2021,['excelente']
1,1,2321,J ’ adore les articles de cette marque,5,2021-06-07,NaN,"Bonjour , Merci infiniment pour votre apprécia...",TrustedShop,ShowRoom,NaN,14.0,2021-06,2021,"['adore', 'articles', 'cette', 'marque']"


In [13]:
# répartion entre variables explicatives et variable cible
X_gb = TrustedShop['comments_filter']
y_gb = TrustedShop['star']

In [14]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_gb, y_gb, test_size=0.2, shuffle=True)

In [15]:
# conversion des chaines de caratères en tokens numériques

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [16]:
# application du modèle de gradientBoosting sur les tokens numériques et calcul des prédictions
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, max_depth=1, learning_rate=1, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [18]:
# calcul des métriques des résultats du modèle
from sklearn.metrics import classification_report

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

# affichage de la matrice de confusion pour voir si les données se sont bien classées
cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['données prédites'])
cm

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.583510261854211

              precision    recall  f1-score   support

           1       0.12      0.04      0.06       135
           2       0.24      0.10      0.14       182
           3       0.36      0.17      0.23       339
           4       0.39      0.27      0.32       683
           5       0.67      0.93      0.78      1487

    accuracy                           0.58      2826
   macro avg       0.35      0.30      0.31      2826
weighted avg       0.51      0.58      0.53      2826



données prédites,1,2,3,4,5
données réelles,,,,,
1,6,12,18,37,62
2,18,18,35,52,59
3,13,33,59,117,117
4,15,11,38,185,434
5,0,1,16,89,1381


In [19]:
print("le modèle prédit bien les bon avis (à 5 star) avec 67% de précision, par contre, plutot mal les notes à 5 star avec seulement 50% et encore pire pour les résultats intermédiaires, on peut penser que le modèle fait de l'overfitting")

le modèle prédit bien les bon avis (à 5 star) avec 67% de précision, par contre, plutot mal les notes à 5 star avec seulement 50% et encore pire pour les résultats intermédiaires, on peut penser que le modèle fait de l'overfitting


In [20]:
# on peut tester le modèles sur de nouveaux commentaire pour voir la prédictions qui seraits faites

new_comments = ["Satisfait et delai de livraison au top", "Bel article mais malheureusement trop petit", "Très déçu et c'est la deuxième fois ! ! ! Il n ' y avait aucun produits qui correspondaient à ma commande et en plus ils étaient cassés", "La veste est trop serré 42 veste Que j ’ avais déjà commander en 40 Déçu Et plus de taille au dessus du 42 dommage Pour cet article"]

# on tokenize les commentaires à partir du modèle vectorizer existant
new_comments_token = vectorizer.transform(new_comments).todense()

# on prédit les données à partir à parti du modèle clf et on regarde si c'est cohérent
y_predict = clf.predict(new_comments_token)
print(y_predict)

[5 4 5 3]


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [ ]:
print("les prédictions ne sont clairement pas satisfaisantes")